# PyTao Plotting

---
## Tao setup

In [ ]:
from pytao import Tao
from pytao.tao_ctypes import util
from pytao.tao_ctypes.util import parse_bool
import os

In [ ]:
# Point to local installation
BASE_DIR=os.environ['ACC_ROOT_DIR'] 
print('Bmad installation: ', BASE_DIR)


# Pick an example init
root = os.path.join(BASE_DIR,'tao/examples/cesr/')
os.chdir(root)
init = root+'tao.init'
# Make tao instance
tao=Tao()   
tao.init('-noplot -init '+init);

In [ ]:
# Enable plotting, and put assign plot 'r1' according to the 'beta' template
tao.cmd('set global force_plot_data_calc = T')
tao.cmd('place r1 beta')
tao.cmd('set plot r1 visible = T')

---
## plot_list

In [ ]:
# Index, region, template, visible 
tao.cmd('python plot_list r')[0:10]

In [ ]:
def parse_plot_regions(lines):
    regions = []
    for line in lines:
        ix, region, plot, visible = line.split(';')
        if plot =='':
            plot = None
        regions.append( {'ix':int(ix), 'region':region, 'plot':plot, 'visible':parse_bool(visible)} ) 
    return regions

def plot_regions(tao):
    return parse_plot_regions(tao.cmd('python plot_list r'))
# list of visible regions
[p for p in plot_regions(tao)  if p['visible'] ]

In [ ]:
# index, template
tao.cmd('python plot_list t')[0:10]

In [ ]:
def parse_plot_templates(lines):
    templates = []
    for line in lines:
        ix, template = line.split(';')
        templates.append(template)
    return templates
def plot_templates(tao):
    return parse_plot_templates(tao.cmd('python plot_list t'))
templates = plot_templates(tao)
templates[0:10]

## plot1 

In [ ]:
# name, type, settable, value
tao.cmd('python plot1 r1')

In [ ]:
util.parse_tao_python_data(tao.cmd('python plot1 r1'))

---
## plot_graph 

In [ ]:
tao.cmd('python plot_graph r1.a')

In [ ]:
util.parse_tao_python_data(tao.cmd('python plot_graph r1.a'))

---
## plot_curve

In [ ]:
tao.cmd('python plot_curve r1.a.c1')

In [ ]:
util.parse_tao_python_data(tao.cmd('python plot_curve r1.a.c1'))

---
## plot_line

In [ ]:
tao.cmd('python plot_line r1.a.c1')[0:10]

In [ ]:
# X array
tao.cmd_real('python plot_line r1.a.c1 x')[0:10]

In [ ]:
# Y array
tao.cmd_real('python plot_line r1.a.c1 y')[0:10]

In [ ]:
def parse_plot_line(lines):
    xlist = []
    ylist = []
    for line in lines:
        i, x, y = line.split(';')
        xlist.append(float(x))
        ylist.append(float(y))
    return {'x':xlist, 'y':ylist}
parse_plot_line(tao.cmd('python plot_line r1.a.c1')).keys()

---
## plot_symbol 

In [ ]:
tao.cmd('python plot_symbol r1.a.c1')[0:10]

In [ ]:
def parse_plot_symbol(lines):
    xlist = []
    ylist = []
    ixlist = []
    for line in lines:
        i, ix, x, y = line.split(';')
        ixlist.append(int(ix))
        xlist.append(float(x))
        ylist.append(float(y))
    return {'x_symbol':xlist, 'y_symbol':ylist, 'ix_symbol':ixlist}

In [ ]:
symbol_lines = tao.cmd('python plot_symbol r1.g.a')

In [ ]:
# X array
tao.cmd_real('python plot_symbol r1.a.c1 x')[0:10]

In [ ]:
# Y array
tao.cmd_real('python plot_symbol r1.a.c1 y')[0:10]

---
## Classes

In [ ]:
class TaoCurve:
    def __init__(self, tao, identifier):
        self.identifier = identifier # region.graph.curve, like: r1.g.c1
        self.update()
        
    def update(self):
        self.__dict__.update(util.parse_tao_python_data(tao.cmd('python plot_curve '+self.identifier)))
        # Get points
        self.data = {}
        if self.draw_line:
            self.data['x'] = tao.cmd_real('python plot_line '+self.identifier+ ' x')
            self.data['y'] = tao.cmd_real('python plot_line '+self.identifier+ ' y')
            #self.data.update(parse_plot_line(tao.cmd('python plot_line '+self.identifier)))
        if self.draw_symbols:
            self.data['x_symbol'] = tao.cmd_real('python plot_symbol '+self.identifier+ ' x')
            self.data['y_symbol'] = tao.cmd_real('python plot_symbol '+self.identifier+ ' y')
            #self.data.update(parse_plot_symbol(tao.cmd('python plot_symbol '+self.identifier)))
          
    def __str__(self):
        return 'Tao curve '+self.identifier+' with '+str(len(self.data['x']))+' points'

In [ ]:
class TaoGraph:
    def __init__(self, tao, identifier):
        self.identifier = identifier # region.graph, like: r1.g
        self.update()
        
    def update(self):
        self.__dict__.update(util.parse_tao_python_data(tao.cmd('python plot_graph '+self.identifier)))
        self.curve = []
        for i in range(self.num_curves):
            # Clean up keys of the form curve[1]
            key = 'curve['+str(i+1)+']'
            curve_name = self.__dict__[key]
            del(self.__dict__[key])
            # Add curve
            curve = TaoCurve(tao, self.identifier+'.'+curve_name)
            self.curve.append(curve)    
            
    def __str__(self):
        cnames = ''
        for c in self.curve:
            cnames=cnames+c.name+' '
        return 'Tao graph '+self.identifier+' with '+str(len(self.curve))+' curves: '+cnames

In [ ]:
class TaoPlot:
    def __init__(self, tao, region):
        self.identifier = region
        self.update()
        
    def update(self):
        self.__dict__.update(util.parse_tao_python_data(tao.cmd('python plot1 '+self.identifier)))
        self.graph = []
        for i in range(self.num_graphs):
            # Clean up keys of the form: graph[1].
            key = 'graph['+str(i+1)+']'
            graph_name = self.__dict__[key]
            del(self.__dict__[key])
            # Add graph
            graph = TaoGraph(tao, self.identifier+'.'+graph_name)
            self.graph.append(graph)
            
    def __str__(self):
        gnames = ''
        for g in self.graph:
            gnames=gnames+g.name+' '
        return 'Tao plot '+self.identifier+' with '+str(len(self.graph))+' graphs: '+gnames
plot = TaoPlot(tao, 'r1')

In [ ]:
plot.graph[0].curve[0].__dict__.keys()

In [ ]:
print(plot)
print(plot.graph[0])
print(plot.graph[0].curve[0])

In [ ]:
plot.graph[0].curve[0].data.keys()

In [ ]:
list(plot.__dict__.keys())

In [ ]:
list(plot.graph[0].__dict__.keys())

---

## Simple Bokeh plotting example

In [ ]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.models.sources import ColumnDataSource
output_notebook()

In [ ]:
# Set up a plot
tao.cmd('set global force_plot_data_calc = T')
tao.cmd('place r1 beta')
tao.cmd('set plot r1 visible = T')
tao.cmd('x-axis r1 s')
tao.cmd('x-s r1 0 1000')


In [ ]:
# Simple function to parse the curve data
def simple_parse_cuve_data(lines, xlabel='x', ylabel='y'):
    xdat = []
    ydat = []
    for line in lines:
        ix, x, y = line.split(';')
        xdat.append(float(x))
        ydat.append(float(y))
    return {xlabel:xdat, ylabel:ydat}
curve_dat = simple_parse_cuve_data(tao.cmd('python plot_line r1.a.c1'))
#curve_dat.update(simple_parse_cuve_data(tao.cmd('python plot_line r1.b.c1'))

In [ ]:
# create a new plot with a title and axis labels
p = figure(title="Beta plot", x_axis_label='s', y_axis_label='beta_a')

source = ColumnDataSource(data=curve_dat)

# add a line renderer with legend and line thickness
p.line('x', 'y', legend="Temp.", line_width=2, source=source)

# show the results
show(p)

In [ ]:
p2 = figure(title="Beta plot", x_axis_label='s', y_axis_label='beta_a')
source2 = ColumnDataSource(data=plot.graph[0].curve[0].data)
p2.line('x', 'y', legend="Temp.", line_width=2, source=source2)
show(p2)

---
## Full Bokeh 

In [ ]:
list(plot.graph[0].__dict__.keys())

In [ ]:
list(plot.graph[0].curve[0].__dict__.keys())

In [ ]:
%%tao
set plot r1 n_curve_pts = 10

In [ ]:
from bokeh.colors import named
def bokeh_color(qp_color):
    """
    Try to get a named color
    """
    return named.__dict__[qp_color.lower()]

def bokeh_graph(tao_graph):
    g = tao_graph
    p = figure(title=g.title, x_axis_label=g.x_label, y_axis_label=g.y_label)
    
    #p.x_range.on_change('start', lambda attr, old, new: print("Start", new))
    for c in g.curve:
        source = ColumnDataSource(c.data)
        if c.draw_line:
            p.line('x', 'y', legend=c.data_type, line_width=c.line_width, 
                line_color = c.line_color.lower(), source=source)
        if c.draw_symbols:
            p.circle('x_symbol', 'y_symbol', color=c.symbol_color.lower(), source=source)
    
    return p
tao.cmd('set plot r1 n_curve_pts= 100')
plot.update()
show(bokeh_graph(plot.graph[0]))
    

In [ ]:
templates = plot_templates(tao)
def test_template(t):
    tao.cmd('place r1 '+t)
    tao.cmd('set plot r1 visible = T')
    plot.update()
    return bokeh_graph(plot.graph[0])
for t in templates:
    print(t)
    try:
        show(test_template((t)))
    except:
        pass
pout = test_template(templates[0])    
#export_png(pout, 'test.png')

---
## Test

In [ ]:
def test2_template(t):
    print('place r1 '+t)
    print('set plot r1 visible = T')

In [ ]:
for t in templates:
    try:
        test2_template((t))
    except:
        pass